In [ ]:
from video_commands import *
main_video = "B0815_subtitle.mov"
width,height = get_media_dimensions(main_video)
print(width,height)

## check dimensions of a file

In [ ]:
width,height = get_media_dimensions("black.mov")
print(width,height)



## trim Video
outvideo = "test.mov"<br>
invideo = "B0815.mov"<br>
starttime,endtime = "0:10" , "0:18"<br>

In [ ]:
from video_commands import *
#outvideo = "test_5min.mov"
#invideo = "B0815_2flower.mov"
#starttime,endtime = "17:10" , "20:10"
#trim_video(invideo, "16:30" , "21:30", outvideo)

trim_video("LETTER.mp4", "00:00" , "00:15", "letter_tmp.mov")
trim_video("黄花菜地.mov", "00:02" , "00:22", "黄花菜地_tmp.mov")
trim_video("war.mov", "00:00" , "00:10", "war_tmp.mov")
x, muted_video1, z = separate_audio_video("letter_tmp.mov",".",0)
x, muted_video2, z =separate_audio_video("黄花菜地_tmp.mov",".",0)
x, muted_video3, z =separate_audio_video("war_tmp.mov",".",0)
a_list = [muted_video1, muted_video2, muted_video3]
a,b=reencode_to_match(outvideo,a_list,crf="30",preset="veryfast")
print(a)
print_media_info(a)


## add a black still and insert in video<br>
input_video = "test_1min.mov"<br>
outfile = "test.mov"<br>
insert_black_stills(input_video=input_video,black_list=black_list,output_file=outfile,crf=18,preset="fast")



# # ADD multiple black stills and insert into video
<font color = "red">add def, insert multiple new black stills using concat. Ask me to delete original mov</font>



In [ ]:
from video_commands import *
input_video = "test_5min.mov"
start_time = 0#"17:00.5" #5
end_time = 5#"17:03.5"   #10
crf = 10
preset = "fast"
input_file=[["28年后的巨变","black1.mov"],["日本留学","black2.mov"],["回国梦","black3.mov"]]
#for i in input_file:
#    create_black_still(input_video, i[0],start_time , end_time, i[1], font_name="Arial",font_size=130,font_color="&H00FFFFFF")

video_duration = get_video_length(input_video)
sections = ["0:00-1:00", "1:00-2:00", "2:00-3:00",f"3:00-{video_duration}"]
input_file = "test_5min.mov"
outputfiles = ["video1.mov","video2.mov","video3.mov","video4.mov"]
#for section, outfile in zip(sections, outputfiles):
#     split_video(input_file, [section], [outfile])
video_list =["video1.mov","black1.mov","video2.mov","black2.mov","video3.mov","black3.mov","video4.mov"]
#combine_video(video_list,primary_video_index= ,output_file="test.mov", crf=40, preset="ultrafast")
combine_video(video_list,0,output_file="test_final3.mov", crf=30, preset="veryfast")

In [ ]:
from video_commands import * 
success, temp_muted, audio_file = separate_audio_video("war_muted.mov",".",0)
main_video = "test_3min.mov"
list_to_reencode = ['war_muted.mov']
a,b = reencode_to_match(main_video, list_to_reencode,crf="18",preset="fast")
print(a) # names of reencoded videos
print(b) #dictionary with original and reencoded name
a = ["tmp/test_3min_reencoded_padded.mov","tmp/rescaled_clip_reencoded_padded.mov"]
print_media_info(a)
a = get_video_length("test_3min.mov")
print(a)


In [ ]:
from video_commands import *


## Audio - converting mp3 to wav<br>
mp3_file = "music/darkness.mp3"

In [ ]:
mp3_file = "music/darkness.mp3"
mp3_to_wav(mp3_file)

## Audio - adjusting volume<br>
wav_file = "music/darkness.wav" <br>
volume_factor = 0.1

In [ ]:
wav_file = "music/darkness.wav"
volume_factor = 0.1
adjust_wav_volume(wav_file, volume_factor)

## split video into sections<br>
sections = ["0:00-5:00", "5:00-8:00", "9:00-12:00"]<br>
split_video("xxx.mov", sections)<br>
output:test_5min_0005-0100.mov,test_5min_0100-0200.mov,test_5min_0220-0300.mov


In [ ]:
sections = ["0:05-1:00", "1:00-2:00", "2:20-3:00"]
input_file = "test_5min.mov"
split_video(input_file, sections)

## extract voice from a video and convert to srt file
lang can be None


In [ ]:
input_video, output_srt,lang = "test_1min.mov", "test_1min.srt","zh"
voice_to_srt(input_video, output_srt, lang=lang)#en,ja,k

## srt to ass

In [ ]:
srt_file = "test_1min.srt" #input file
ass_file = "test_1min.ass" #new file
srt_to_ass(srt_file, ass_file,width,height,fontname="宋体", fontsize=30)
burn_subtitles("test_1min.mov", "test_1min.ass", "output.mov", "Arial")


## Translation and Combine Different Translations for proofreading
<h3>only for srt!!!</h3>

In [ ]:
inputfile,outputfile,lang = "test_1min.srt","test_1min_jp.srt","JA" #lang = target language
deepl_translate_srt(inputfile, outputfile ,lang=lang)#EN-US KO zh JA

In [ ]:
inputfile = "test_1min.srt"
outputfile = "test_1min_jp.srt"
combined_file = "combined_chi_jp.srt"  #new file, oneline chinese one line japanese
combine_srt(inputfile,outputfile, combined_file)

## Add grid to a screenshot for mosaic
add_numbered_grid(<br>
    input_file,output_file,width,height,interval=100,line_color="red",<br>
    number_color="yellow",number_size=30,number_offset=5)

In [ ]:
time_and_name = [
        ["1:10", "test1.png"],           # 1 minute 10 seconds
        ["70", "test2.jpg"],             # 70 seconds
        ["1:30.5", "test3.png"],         # 1 minute 30.5 seconds
        ["00:01:45", "test4.jpg"],       # 1 minute 45 seconds (full format)
        ["00:02:30.750", "test5.png"],   # 2 minutes 30.75 seconds
        ["4:41", "test6.jpg"]            # 4 minutes 41 seconds
        ]
extract_frames("test_5min.mov", time_and_name)

In [ ]:
for i in time_and_name:
    outscreenshot = f"{i[1].split(".")[0]+"_grid.png"}"
    add_numbered_grid(
    i[1],
    outscreenshot,
    width,
    height,
    interval=50,
    line_color="red",
    number_color="yellow",
    number_size=30,
    number_offset=5
)

# Apply Mosaic

## example: <br>
mosaic_list=[<br>
        [50, 50, 100, 50, 10, 0, 10],      <br>
        [800, 0, 200, 200, 15, "1:20.5", "3:00"]]  <br>
    # position of top-left corner<br>
    x=50,y=50,width=100,height=50<br>
    pixelization level=10<br>
    start seconds=0,end seconds=10<br>

In [ ]:
input_video,output_video = "test_5min.mov","test_5min_mosaic.mov"
mosaic_list = [[65, 50, 200, 170, 15, 0, 30],[width-260, 60, 285, 140, 15, "1:20.5", "2:30"]]
apply_mosaics(input_video, output_video, mosaic_list)

## srt to ass and burn in subtitles <br>
srt_file = "../中文字幕.srt" #input file <br>
ass_file = "中文字幕.ass" #new file <br>
input_video = "B0815_4letter.mov" <br>
output_video = "B0815_subtitle.mov" <br>
#srt_to_ass(srt_file, ass_file,width,height,fontname="宋体", fontsize=30) <br>
burn_subtitles(input_video, ass_file, output_video, "Arial",crf ="18",preset="veryfast") <br>

In [ ]:
srt_file = "中文字幕.srt" #input file
ass_file = "中文字幕.ass" #new file
input_video = "B0815_4letter.mov"
output_video = "B0815_subtitle.mov"
srt_to_ass(srt_file, ass_file,width,height,fontname="宋体", fontsize=30)
burn_subtitles(input_video, ass_file, output_video, "Arial",crf ="10",preset="fast")

In [ ]:
srt_file = "test_1min.srt" #input file
ass_file = "test_1min.ass" #new file
input_video = "test_1min.mov"
output_video = "test.mov"
srt_to_ass(srt_file, ass_file,width,height,fontname="宋体", fontsize=30)
burn_subtitles(input_video, ass_file, output_video, "Arial")

## End film
input "end_subtitle_txt" in the end film this way:<br>
编导演播：翰 光<br>
音乐：Chad Cannon<br>
照片提供： 小林熏<br>
         李晓斌<br>

The content of <b>end_subtitle_txt</b> is inserted into <b>end_subtitle_template</b> <br>
as <b>end_subtitle_ass</b> by <b><i>end_credits_ass</b></i>.<br>
Then <b><i>create_ending_film</b></i> is called to create <b>end_output</b> with <b>end_song</b> as background music.<br>
The dimension of  <b>end_output</b> is the same as the <b>main_video</b>. The background music has an adjustable volume<br>
and fade-in,fade-out time. <b>end_second</b> is the silent time before the end of the film and after the end of the song.
You can add a background video with an adjustable fade-out time. <br>
Remember to use <b><i>preprocess</b></i> to rescale it first.<br>
Add rescale to match the scale

In [ ]:
from video_commands import *
width,height = get_media_dimensions("test_3min.mov")
print(width,height)

end_subtitle_ass= "2508credits2.ass"
end_subtitle_txt= "end_subtitle.txt"
end_subtitle_template = "end_subtitle_template.ass"
end_song = "music/end_song.wav"
end_output3 = "end_vol4.mov"
end_credits_ass2(end_subtitle_template, end_subtitle_txt, end_subtitle_ass,width, height,first_start="0:00:01.40", 
                 each_duration_s=6.0,line_offset_s=0.60, fallback_duration_s=4.0)
create_ending_film(end_subtitle_ass, end_song, end_output3, width, height,
                       volume=3, end_second=5, fadein=3, fadeout=10,
                       bg_video=None,video_fadeout=4)

## Reencode and combine 
Do the reencoding inside
output_file="B0815_final1.5.mp4" not ok,must be <br>
output_file="B0815_final1.mp4", no "." in between

In [ ]:
from video_commands import *
#main_video = "test_3min.mov"
#width,height = get_media_dimensions(main_video)
list_to_reencode = ["beginning_ban.mp4","end_vol3.mov"]
video_list = ["letter.MP4","test_3min.mov","end_vol4.mov"]
#make_rescaled_image(width, height, "beginning_ban.mp4", "beginning_ban_rescaled.mp4")
#make_rescaled_image(width, height, "end_vol3.mov", "end_vol3_rescaled.mov4")
#a,b= reencode_to_match2(main_video, list_to_reencode)
#print(a)
#print(b)
#files = ["tmp/1_test_3min_reencoded.mov","tmp/0_beginning_ban_reencoded.mov","tmp/2_end_vol3_reencoded.mov"]
#print_media_info(["end_vol4.mov"])
#a,b =reencode_to_match("test_3min.mov",["end_vol4.mov"],purpose="combine",crf=40,preset="ultrafast")
#make_rescaled_image(854, 480,"end_vol4.mov", "combine","test_rescaled.mov",crf=30,preset="ultrafast")
"""
a=["test_3min_reencoded_padded.mov","end_vol4_reencoded_padded.mov"]
print_media_info(a)
a,b = get_media_dimensions("end_vol4_reencoded_padded.mov")
print(a,b)
a,b = get_media_dimensions("end_vol4.mov")
print(a,b)
"""
combine_video(video_list, primary_index=1,output_file="B0815_final_3min.mov", crf=30,preset="ultrafast")
#reencode_to_match(primary_video, list_to_reencode,purpose="combine",crf=crf, preset=preset)
#make_rescaled_image(video_width, video_height, image_name, purpose,output_filename,crf=18,preset="fast"):

In [ ]:
#complicated method
time1,time2 = "13:09.8","13:14.9"
duration = parse_time(time2)-parse_time(time1)
print(duration)
overlay_video_split_and_combine(
    main_video="B0815_3flower.mov",
    overlay_video="video_folder/letter_muted.mp4",
    overlay_start="0:02",         # trim overlay from 0:02-0:02+duration
    overlay_duration = duration,        # modify to the end
    main_insert_at= time1,    
    output_file="B0815_4letter.mov",# insert overlay at 5:00 of main
    crf=10,
    preset="fast",
    keep_intermediates=True
)



In [ ]:
make_rescaled_image(width, height, "班和保人.png", "test.png")

In [ ]:
from video_commands import *
main_video = "test_3min.mov"
output_file = "overlay_test2.mov"
para_list = [["letter.MP4","1:30","1:45","0:00","2"],["war.mov","2:00","2:20","0:02","0"],["taiwan.MP4","2:40","2:50","0:01","5"],
            ["残留夫人.png","1:50","1:55","0:00","1"],["班和保人.png","2:30","2:35","0:00","2"]]
    
overlay_video_and_img(main_video,para_list,output_file,crf="30",preset="veryfast")

1.insert multiple images to overlay_video<br>
2.overlay a single image, or list image<br>
3.overlay_black_still not working in github<br>
4. 更改片尾字幕resolution<br>
5.overlay_music<br>
6. use the following to check real video size to adjust rescaling for vertical videos<br>
ffmpeg -i video.mp4 -vf "cropdetect=24:16:0" -frames:v 1 -f null -<br>
or<br>
import cv2<br>
import numpy as np<br>
<br>
cap = cv2.VideoCapture("video.mp4")<br>
ret, frame = cap.read()<br>
if ret:<br>
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)<br>
    # Count non-black pixels<br>
    non_black = np.sum(gray > 16)<br>
    total_pixels = gray.size<br>
    # Simple ratio of non-black region<br>
    ratio = non_black / total_pixels<br>
    print("Non-black ratio:", ratio)<br>
cap.release()<br>



from video_commands import *
a,b = get_media_dimensions("xingxing.MP4")
c,d = get_media_active_dimensions(file_path)

<h3> ***continue here , burn in subtitle </h3>

In [ ]:
from video_commands import *
video_width, video_height = get_media_dimensions("test_3min.mov")
"""
infile="黄花菜地.mov"
infile = "xingxing.mp4"
c,d = get_media_active_dimensions(infile)
print("main_video dimensions: ",video_width, video_height)
print(f"{infile} active dimensions: ",c,d)
a,c = get_media_dimensions(infile)
print(f"{infile} dimensions: ",a,c)
#make_rescaled_image(video_width, video_height, infile, "test_rescaled.MP4")
#make_rescaled_image2(video_width, video_height, infile, "test_rescaled.MP4")
a,b = reencode_to_match("test_1min.mov", [infile], crf="23", preset="fast")
print_media_info(a)
"""
make_rescaled_image(video_width, video_height, "残留夫人.png", "combine", "残留夫人_rescaled.png")
add_numbered_grid(
    "残留夫人_rescaled.png",
    "残留夫人_grid.png",
    video_width,
    video_height,
    interval=50,
    line_color="red",
    number_color="yellow",
    number_size=30,
    number_offset=5)

In [ ]:
from video_commands import *
main_video = "test_3min.mov"
output_file = "overlay_test3.mov"
img_name,x_pos,y_pos,img_duration,zoom_start,zoom_max,out_file = "班和保人.png","620","200","0:05","0:01","2","班和保人.mov"
img_name2,x_pos2,y_pos2,img_duration2,zoom_start2,zoom_max2,out_file2 = "残留夫人.png","400","170","0:05","0:01","2","残留夫人.mov"
image_list = [[img_name,x_pos,y_pos,img_duration,zoom_start,zoom_max,out_file],
              [img_name2,x_pos2,y_pos2,img_duration2,zoom_start2,zoom_max2,out_file2]]
#create_zoom(image_list, main_video, crf=23, preset="veryfast")

para_list = [["班和保人.mov","0:30","0:35","0:00","0"],["黄花菜地.mov","1:00","1:20","0:00","2"],["letter.MP4","2:00","2:10","0:05","2"],
             ["班和保人.png","2:30","2:35","0"],["残留夫人.png","2:45","2:50","2"]]
#test with only 1 video, and i image, then change img2 to 1
overlay_video_and_img2(main_video,para_list,output_file,crf="30",preset="ultrafast")

para_list = [["班和保人.mov","0:30","0:35","0:00","0"],["黄花菜地.mov","1:00","1:20","0:00","2"],["letter.MP4","2:00","2:10","0:05","2"]]
#test with only 1 video, and i image, then change img2 to 1
overlay_video_and_img2(main_video,para_list,"test_vidoe_only.mov",crf="30",preset="ultrafast")

para_list = [["班和保人.png","2:30","2:35","0"],["残留夫人.png","2:45","2:50","2"]]
#test with only 1 video, and i image, then change img2 to 1
overlay_video_and_img2(main_video,para_list,"test_img_only.mov",crf="30",preset="ultrafast")

para_list = [["黄花菜地.mov","1:00","1:20","0:00","2"],]
#test with only 1 video, and i image, then change img2 to 1
overlay_video_and_img2(main_video,para_list,"test_one_video.mov",crf="30",preset="ultrafast")

para_list = [["残留夫人.png","2:45","2:50","2"]]
#test with only 1 video, and i image, then change img2 to 1
overlay_video_and_img2(main_video,para_list,"test_one_img.mov",crf="30",preset="ultrafast")

end_time = get_video_length(main_video)
end_time = parse_time(end_time)
end_start = end_time - 10
para_list = [["黄花菜地.mov",end_start,end_time,"0:5","2"]]
#test with only 1 video, and i image, then change img2 to 1
overlay_video_and_img2(main_video,para_list,"test_end.mov",crf="30",preset="ultrafast")





    cmd = [
        "ffmpeg",
        "-y",
        "-ss", start_time,
        "-to", end_time,
        "-i", input_file,
        "-c:v", "libx264",
        "-preset", "fast",
        "-crf", "18",
        "-c:a", "copy",
        output_file
]

In [ ]:
from video_commands import *
main_video = "test_3min_final.mov"
output_file = "overlay_test.mov"
para_list = [ ["黄花菜地.mov","2:30","2:35","0:00","2"],
              ["letter.MP4","3:00","3:15","0:00","2"],
             ["班和保人_rescaled.png","2:30","2:35","2"]]
    
overlay_video_and_img(main_video,para_list,output_file,crf="40",preset="ultrafast")

In [ ]:
from video_commands import *
images = [
    ["班和保人.png","620","200","5.0","1.5","2","output1.mov"],
    ["残留夫人.png","300","100","5.0","1.0","1.5","output2.mov"]
]
create_zoom(images, main_video="test_3min_final.mov", crf=20, preset="veryfast")
a=["output1.mov","output2.mov"]
print_media_info(a)

In [ ]:
add_numbered_grid(
    "班和保人_rescaled.png",
    "班和保人_grid.png",
    854,
    480,
    interval=50,
    line_color="red",
    number_color="yellow",
    number_size=30,
    number_offset=5)

## notes
reencode_to_match()<br>
reencode videos in aac, but supposed to be in pcm

In [ ]:
trim soong from 30 s to 60s
ffmpeg -ss 30 -i xxx.wav -t 30 -c copy xxx2.wav



# correct ,copy to markdown<br>
ffmpeg -i test_3min_final.mov -i 黄花菜地_final.mov -i letter_final.mov -i 班和保人_rescaled.png \
-filter_complex "
color=black:size=854x480:d=5[black1];
color=black:size=854x480:d=15[black2];
[1:v]trim=0:5,setpts=PTS-STARTPTS,format=rgba,fade=t=out:st=3:d=2:alpha=1[ov1];
[2:v]trim=0:15,setpts=PTS-STARTPTS,format=rgba,fade=t=out:st=13:d=2:alpha=1[ov2];
[black1][ov1]overlay=x=(W-overlay_w)/2:y=(H-overlay_h)/2:shortest=1[ov_final1];
[black2][ov2]overlay=x=(W-overlay_w)/2:y=(H-overlay_h)/2:shortest=1[ov_final2];
[3:v]loop=loop=-1:size=1:start=0,setpts=PTS-STARTPTS,format=rgba,scale=w=min(iw\,854):h=min(ih\,480):force_original_aspect_ratio=decrease,pad=854:480:(ow-iw)/2:(oh-ih)/2:color=black,fade=t=in:st=150:d=0:alpha=1,fade=t=out:st=153:d=2:alpha=1[img1];
[0:v]trim=0:150,setpts=PTS-STARTPTS[main_pre1];
[0:v]trim=150:155,setpts=PTS-STARTPTS[main_mid1];
[0:v]trim=155:180,setpts=PTS-STARTPTS[main_pre2];
[0:v]trim=180:195,setpts=PTS-STARTPTS[main_mid2];
[0:v]trim=195,setpts=PTS-STARTPTS[main_post];
[main_mid1][ov_final1]overlay[main_ov1];
[main_mid2][ov_final2]overlay[main_ov2];
[main_pre1][main_ov1][main_pre2][main_ov2][main_post]concat=n=5:v=1:a=0[vout];
[vout][img1]overlay=enable='between(t,150,155)':shortest=1[vout_final]
" -map "[vout_final]" -map 0:a -c:v libx264 -pix_fmt yuv420p -preset ultrafast -crf 40 -c:a aac -b:a 192k -movflags +faststart -y overlay_test.mov

In [ ]:
music/end_vol30.mov","1:30","2:00","0:00","2"  #start time, end time, fade in, fade out , volume factor

In [ ]:
from video_commands import *
main_video = "test_3min.mov"
output_file = "overlay_com.mov"
para_list = [["黄花菜地.mov","0:30","0:50","0:02","2"],["war.mov","1:00","1:30","0:02","2"],["letter.MP4","1:40","1:55","0:02","2"],
             ["残留夫人.png","1:30","1:35","2"],["班和保人.png","1:40","1:45","2"],["曾大爷.png","2:00","2:10","2"],
             ["music/end_song30.wav","1:00","2:00","2","2","0.2"], ["music/end_song.mp3","2:30","3:00","2","2","0.2"]]
#["黄花菜地.mov","0:30","0:50","0:02","2"],    "0:02 "黄花菜地.mov start time
overlay_video_img_music(main_video,para_list,output_file,crf="30",preset="ultrafast")
#"music/end_song.mp3","1:00","2:00","2","2","0.2 2=fadein and fadeout , 0.2 = volume factor

In [1]:
from video_commands import *
main_video = "test_3min.mov"
output_file = "overlay_1_music.mov"
para_list = [["music/end_song.mp3","2:30","3:00","2","2","0.2"]]
    
overlay_video_and_img3(main_video,para_list,output_file,crf="30",preset="ultrafast")

temp_dir: /var/folders/9g/509r85b95jz43885pc6n7zlr0000gn/T/tmpeqsn63hx


ffmpeg version 8.0 Copyright (c) 2000-2025 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.1.0.2.5)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/8.0 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enabl

Converted: music/end_song.mp3 -> music/end_song.wav
⚡ Re-encoding audio to PCM: test_3min.mov → test_3min_tmp.mov


ffmpeg version 8.0 Copyright (c) 2000-2025 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.1.0.2.5)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/8.0 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enabl

new width height: 854 480
FFmpeg command: ffmpeg -y -i test_3min_tmp.mov -vf scale=854:480,pad=854:480:0:0:black -c:v libx264 -preset ultrafast -crf 30 -c:a copy test_3min_reencoded.mov
ffmpeg version 8.0 Copyright (c) 2000-2025 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.1.0.2.5)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/8.0 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --en

ffmpeg version 8.0 Copyright (c) 2000-2025 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.1.0.2.5)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/8.0 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enabl

✅ Final re-encoded + padded file: test_3min_reencoded_padded.mov
Generated ffmpeg command:
 ffmpeg -i /var/folders/9g/509r85b95jz43885pc6n7zlr0000gn/T/tmpeqsn63hx/test_3min_reencoded_padded.mov \
-stream_loop -1 -i /var/folders/9g/509r85b95jz43885pc6n7zlr0000gn/T/tmpeqsn63hx/music_0.wav \
-filter_complex "
[0:v]trim=0,setpts=PTS-STARTPTS[main_post];
 [main_post]concat=n=1:v=1:a=0[vout];
 [1:a]atrim=0:30.0,asetpts=PTS-STARTPTS,volume=0.2,afade=t=in:st=0:d=2.0,afade=t=out:st=28.0:d=2.0,adelay=150000|150000[a1];
 [0:a][a1]amix=inputs=2:normalize=0[aout]
" -map "[vout]" -map "[aout]" -c:v libx264 -pix_fmt yuv420p -preset ultrafast -crf 30 -c:a aac -b:a 192k -movflags +faststart -y overlay_1_music.mov


ffmpeg version 8.0 Copyright (c) 2000-2025 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.1.0.2.5)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/8.0 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enabl